In [ ]:
# setup RPA environment by installing PHP, Chromium web browser and RPA for Python
!apt-get update; apt install php; apt install chromium-browser; pip install rpa

# run r.setup() to download TagUI, modify to run Chromium browser and as root user
import rpa as r; r.setup(); r.dump(r.load('/root/.tagui/src/tagui').replace('"google-chrome"', '"chromium-browser"').replace('$headless_switch', '--no-sandbox'), '/root/.tagui/src/tagui')

# virtual display to run web browser normally (some websites reject headless mode)
!apt install xvfb; pip install pyvirtualdisplay
import pyvirtualdisplay; display = pyvirtualdisplay.Display(); display.start()

# Actualizamos las librerias 
!pip install --upgrade pandas
!pip install --upgrade xlrd
!pip install --upgrade openpyxl

In [2]:
import pandas as pd
import rpa as r
import time
import os

In [3]:
# Conectamos Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Leemos el archivo
import pandas as pd
cui = pd.read_csv('/content/drive/MyDrive/DATA HUNTERS - DSRP 2022/1.- BASE DE DATOS/CUIs/CUI_2019_dep8_cusco.csv', encoding='latin-1')
cui0 = cui.rename(columns = {'PRODUCTO_PROYECTO'  : 'cui',
                             '"PRODUCTO_PROYECTO"': 'cantidad'})

# Registros por region
# Cusco     : 4458
# Piura     : 2309
# MadreDios :  497

cui1 = cui0['cui']
cui2 = cui1[3001:3999]

# Datos para exportar el archivo
rango = "2001_3000"
path_export = "/content/drive/MyDrive/DATA HUNTERS - DSRP 2022/1.- BASE DE DATOS/5.- SSI_cui_ListaEjecucionSimplePublica/cusco/ssi_listaejecucion_2019_"
filetype = ".xlsx"

# Pasamos a una lista en formato string la columna ruc
lista_cui0 = cui2.values.tolist()
lista_cui = list(map(str, lista_cui0))
print(lista_cui)

['2410913', '2411304', '2412227', '2412253', '2412271', '2412594', '2412597', '2412606', '2412609', '2412624', '2412738', '2412927', '2412975', '2413117', '2413156', '2413225', '2413234', '2413456', '2413478', '2413671', '2413688', '2413775', '2413779', '2413782', '2413790', '2413890', '2414158', '2414164', '2414171', '2414197', '2414302', '2414363', '2414375', '2414418', '2414486', '2414568', '2414585', '2414629', '2414859', '2414867', '2415417', '2415541', '2415567', '2415626', '2415737', '2415862', '2415905', '2416109', '2416117', '2416169', '2416182', '2416277', '2416331', '2416341', '2416413', '2416416', '2416421', '2416424', '2416495', '2416509', '2416544', '2416553', '2416571', '2416574', '2416586', '2416614', '2416639', '2416928', '2416991', '2416993', '2417030', '2417033', '2417044', '2417103', '2417131', '2417141', '2417149', '2417160', '2417171', '2417234', '2417378', '2417387', '2417404', '2417542', '2417642', '2417646', '2417648', '2417657', '2417660', '2418225', '2418228'

In [11]:
# Inicializamos RPA
r.init(turbo_mode = True)

True

In [12]:
ssi_cui                      = []
ssi_pip                      = []
ssi_monto_inversion          = []
ssi_monto_actualizado        = []
ssi_fecha_ultimamodificacion = []

# INICIAMOS EL SCRAPEO CON LAS CUI 
for i in range(len(lista_cui)):
    r.url('https://ofi5.mef.gob.pe/invierte/ejecucion/traeListaEjecucionSimplePublica/'+lista_cui[i]+'')
    time.sleep(1)
    
    # Condicional para optimizar la extracción
    element_exist = r.present('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[2]/div[1]')
    print(i, element_exist)
    if element_exist == True:
        
        element_exist1 = r.present('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[2]/div[2]')
        if element_exist1 == True:    
            # scrapeamos la información importante sobre esta ficha
            cui                      = r.read('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[2]/div[2]')
            pip                      = r.read('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]')
            monto_inversion          = r.read('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[4]/div[2]')
            monto_actualizado        = r.read('//*[@id="main-container"]/div[1]/div/div/div/div/div[1]/div[5]/div[2]')
            fecha_ultimamodificacion = r.read('//*[@id="main-container"]/div[1]/div/div/div/div/div[2]/table/tbody/tr/td[1]')

        elif element_exist1 == False:
            cui                      = 0
            pip                      = 0
            monto_inversion          = 0
            monto_actualizado        = 0
            fecha_ultimamodificacion  = 0
 

        # Appendeamos la información extraida en las listas
        ssi_cui.append(cui)
        ssi_pip.append(pip)
        ssi_monto_inversion.append(monto_inversion)
        ssi_monto_actualizado.append(monto_actualizado)
        ssi_fecha_ultimamodificacion.append(fecha_ultimamodificacion)

        time.sleep(1)
    
    elif element_exist == False:
        
        continue
    
r.close()

time.sleep(2)

# Creamos el Dataframe con la información recolectada
ssi_listaejecucion_2019 = pd.DataFrame({"cui"                        : ssi_cui,
                                        "nombre_pip"                 : ssi_pip,
                                        "monto_inversion"            : ssi_monto_inversion,
                                        "monto_actualizado"          : ssi_monto_actualizado,
                                        "fecha_ultimamodificacion"   : ssi_fecha_ultimamodificacion
                                       })

ssi_listaejecucion_2019.to_excel(path_export+rango+filetype, index = False, header=True)

0 False
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
[RPA][ERROR] - cannot find //*[@id="main-container"]/div[1]/div/div/div/div/div[2]/table/tbody/tr/td[1]
32 True
33 True
34 True
[RPA][ERROR] - cannot find //*[@id="main-container"]/div[1]/div/div/div/div/div[2]/table/tbody/tr/td[1]
35 True
36 True
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
51 True
52 True
53 True
54 True
55 True
56 True
57 True
58 True
59 True
60 True
61 True
62 True
63 True
64 True
65 True
66 True
67 True
[RPA][ERROR] - cannot find //*[@id="main-container"]/div[1]/div/div/div/div/div[2]/table/tbody/tr/td[1]
68 True
[RPA][ERROR] - cannot find //*[@id="main-container"]/div[1]/div/div/div/div/div[2]/table/tbody/tr/td[1]
69 True
70 True
71 True
72 True
73 True
7